In [1]:
import numpy as np
import pandas as pd
data_df = pd.read_csv('Merged_Data_202101_202303_v3.csv')
data_df = data_df.iloc[: , 1:]
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412875 entries, 0 to 3412874
Data columns (total 16 columns):
 #   Column                Dtype  
---  ------                -----  
 0   station_id            int64  
 1   Net_Flow              int64  
 2   Split_hour            int64  
 3   is_holiday            bool   
 4   feels_like            float64
 5   weather_Clear         int64  
 6   weather_Clouds        int64  
 7   weather_Drizzle       int64  
 8   weather_Fog           int64  
 9   weather_Haze          int64  
 10  weather_Mist          int64  
 11  weather_Rain          int64  
 12  weather_Smoke         int64  
 13  weather_Snow          int64  
 14  weather_Thunderstorm  int64  
 15  day_of_week           int64  
dtypes: bool(1), float64(1), int64(14)
memory usage: 393.8 MB


In [7]:
raw_df = pd.read_csv('BikeData_202101_202303.csv')
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7183977 entries, 0 to 7183976
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0               int64  
 1   tripduration             int64  
 2   starttime                object 
 3   stoptime                 object 
 4   start station id         int64  
 5   start station name       object 
 6   start station latitude   float64
 7   start station longitude  float64
 8   end station id           int64  
 9   end station name         object 
 10  end station latitude     float64
 11  end station longitude    float64
 12  bikeid                   int64  
 13  usertype                 object 
 14  postal code              object 
dtypes: float64(4), int64(5), object(6)
memory usage: 822.1+ MB


In [38]:
station_df = pd.read_csv('current_bluebikes_stations.csv', header=None)
station_df = station_df.iloc[2:]
station_df.columns = ["Number", "Name", "Lat", "Long", "Seasonal Status", "Municipality", "Total Docks"]
station_df['Lat'] = pd.to_numeric(station_df['Lat'], errors='coerce').round(6)
station_df['Long'] = pd.to_numeric(station_df['Long'], errors='coerce').round(6)
station_df[:5]

,Number,Name,Lat,Long,Seasonal Status,Municipality,Total Docks
2,L32001,Railroad Lot and Minuteman Bikeway,42.416065,-71.153366,Year Round,Arlington,11
3,L32002,Linwood St at Minuteman Bikeway,42.409354,-71.149065,Year Round,Arlington,11
4,L32005,Thorndike Field at Minuteman Bikeway,42.400168,-71.144570,Year Round,Arlington,11
5,L32003,Mass Ave at Grafton St,42.407261,-71.143821,Winter Storage,Arlington,11
6,L32004,Broadway at Grafton St,42.409942,-71.140093,Winter Storage,Arlington,11


In [42]:
station_bikedf1 = raw_df[['start station id', 'start station latitude', 'start station longitude']]
station_bikedf1 = station_bikedf1.rename(columns={
    'start station id': 'Station_ID',
    'start station latitude': 'Lat',
    'start station longitude': 'Long'
})
station_bikedf2 = raw_df[['end station id', 'end station latitude', 'end station longitude']]
station_bikedf2 = station_bikedf1.rename(columns={
    'end station id': 'Station_ID',
    'end station latitude': 'Lat',
    'end station longitude': 'Long'
})
station_bikedf = pd.concat([station_bikedf1, station_bikedf2], axis=0)
station_bikedf['Lat'] = pd.to_numeric(station_bikedf['Lat'], errors='coerce').round(6)
station_bikedf['Long'] = pd.to_numeric(station_bikedf['Long'], errors='coerce').round(6)
print(len(station_bikedf))


14367954


In [43]:
print(len(station_bikedf))
station_bikedf = station_bikedf.drop_duplicates()
print(len(station_bikedf))

14367954
525


In [23]:
print(station_bikedf[station_bikedf['Lat'].isna()])
print(station_bikedf[station_bikedf['Long'].isna()])
print(station_df[station_df['Lat'].isna()])
print(station_df[station_df['Long'].isna()])

print(station_bikedf.dtypes)
print(station_df.dtypes)

Empty DataFrame
Columns: [start station id, Lat, Long]
Index: []
Empty DataFrame
Columns: [start station id, Lat, Long]
Index: []
Empty DataFrame
Columns: [Number, Name, Lat, Long, Seasonal Status, Municipality, Total Docks]
Index: []
Empty DataFrame
Columns: [Number, Name, Lat, Long, Seasonal Status, Municipality, Total Docks]
Index: []
start station id      int64
Lat                 float64
Long                float64
dtype: object
Number              object
Name                object
Lat                float64
Long               float64
Seasonal Status     object
Municipality        object
Total Docks         object
dtype: object


In [47]:
import geopandas
from shapely.geometry import Point

station_bikedf['geometry'] = station_bikedf.apply(lambda x: Point((x['Long'], x['Lat'])), axis=1)
station_df['geometry'] = station_df.apply(lambda x: Point((x['Long'], x['Lat'])), axis=1)

gdf_bike = geopandas.GeoDataFrame(station_bikedf, geometry='geometry')
gdf_station = geopandas.GeoDataFrame(station_df, geometry='geometry')

merged_gdf = geopandas.sjoin(gdf_bike, gdf_station, how="inner", op='intersects')

merged_gdf = merged_gdf.drop(columns=['Lat_right', 'Long_right'])
merged_gdf = merged_gdf.rename(columns={
    'Lat_left': 'Lat',
    'Long_left': 'Long'
})
merged_gdf[:5]

/Users/yuzhuoran/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,Station_ID,Lat,Long,geometry,index_right,Number,Name,Seasonal Status,Municipality,Total Docks
0,91,42.366277,-71.091690,POINT (-71.09169 42.36628),286,M32002,One Kendall Square at Hampshire St / Portland St,Year Round,Cambridge,19
1,370,42.350961,-71.077828,POINT (-71.07783 42.35096),184,D32045,Dartmouth St at Newbury St,Year Round,Boston,19
2,46,42.343666,-71.085824,POINT (-71.08582 42.34367),47,B32005,Christian Science Plaza - Massachusetts Ave at...,Year Round,Boston,19
3,178,42.359573,-71.101295,POINT (-71.10129 42.35957),321,M32041,MIT Pacific St at Purrington St,Year Round,Cambridge,19
4,386,42.368605,-71.099302,POINT (-71.09930 42.36861),343,M32063,Sennott Park Broadway at Norfolk Street,Year Round,Cambridge,19


In [48]:
print(len(merged_gdf))

434


In [49]:
merged_gdf.to_csv('Merged_Station.csv')